# The London Railway Network

The cell below defines the abstract class whose API you will need to impement. Do NOT modify it.

In [1]:
# DO NOT MODIFY THIS CELL

from abc import ABC, abstractmethod  

class AbstractLondonRailwayMapper(ABC):
    
    # constructor
    @abstractmethod
    def __init__(self):
        pass           
        
    # data initialisation
    @abstractmethod
    def loadStationsAndLines(self):
        pass

    # returns the minimum number of stops to connect station "fromS" to station  "toS"
    # fromS : str
    # toS : str
    # numStops : int
    @abstractmethod
    def minStops(self, fromS, toS):     
        numStops = -1
        return numStops    
    
    # returns the minimum distance in miles to connect station "fromS" to station  "toS"
    # fromS : str
    # toS : str
    # minDistance : float
    @abstractmethod
    def minDistance(self, fromS, toS):
        minDistance = -1.0
        return minDistance
    
    # given an unordered list of station names, returns a new railway line 
    # (represented as a list of adjacent station names), connecting all such stations 
    # and such that the sum of the distances (in miles) between adjacent stations is minimised
    # inputList : set<str>
    # outputList : list<str>
    @abstractmethod
    def newRailwayLine(self, inputList):
        outputList = []
        return outputList

Use the cell below to define any data structure and auxiliary python function you may need. Leave the implementation of the main API to the next code cell instead.

In [2]:
# ADD YOUR DATA STRUCTURE DEFINITIONS AND HELPER CODE HERE
class Station:  # class to represent stations and store their attributes
    def __init__(self, ID, name, lat, long):
        self.ID = ID
        self.name = name
        self.lat = lat
        self.long = long

    def updateID(self, newID):  # updates the ID of a station
        self.ID = newID


class TubeMap:
    # tubeMaps will represent our graph, with an adjacency list for every station in the map.
    def __init__(self):
        self.adj = []  # will hold the edges of our graph
        self.stations = []  # will store all the stations(vertices) of our graph

    def idFromName(self, name):
        # binary search applied to strings, to get the ID of the station with the given name.
        lo = 0
        hi = len(self.stations)
        searched_string = name.lower()  # lowercase gets rid of errors comparing strings caused by capitalisation.

        while lo <= hi:
            mid = lo + ((hi - lo) // 2)

            mid_name = self.stations[mid].name.lower()

            if searched_string == mid_name:
                return self.stations[mid].ID

            if searched_string > mid_name:
                lo = mid + 1

            else:
                hi = mid - 1
        return -1

    def stationFromName(self, name):  # binary search to look for a station with the given name in our list of stations
        lo = 0
        hi = len(self.stations)
        searched_string = name.lower()

        while lo <= hi:
            mid = lo + ((hi - lo) // 2)

            mid_name = self.stations[mid].name.lower()

            if searched_string == mid_name:
                return self.stations[mid]

            if searched_string > mid_name:
                lo = mid + 1

            else:
                hi = mid - 1
        return -1

    def containsEdge(self, idFrom, idTo):
        # looks through all the edges in the adjacency list of a station with id "idFrom", to see if there is an edge
        # connecting it to another station with id "idTo"
        for myEdge in self.adj[idFrom]:
            if myEdge.idTo == idTo:
                return True
        return False

    def addEdge(self, idFrom, idTo):
        # given the IDs of two stations, creates an edge connecting them and adds it to the adjacency list of both.
        station1 = self.stations[idFrom]
        station2 = self.stations[idTo]

        weight = haversine(station1, station2)

        if not self.containsEdge(idFrom, idTo):
            self.adj[idFrom].append(Edge(idFrom, idTo, weight))
        if not self.containsEdge(idTo, idFrom):
            self.adj[idTo].append(Edge(idTo, idFrom, weight))


class Edge:
    # edges will store the IDs of the stations they join. It has a weight equal to the straight-line distance
    # between both stations.
    def __init__(self, idFrom, idTo, weight):
        self.idFrom = idFrom
        self.idTo = idTo
        self.weight = weight


def contains(myList, searched):
    # myList is a list of couples, contains() determines whether in the set of the second component of the couple
    # contains the "searched" element.
    for element in myList:
        if element[1] == searched:  # element[1] is the second element of the couple.
            return True
    return False


def updatePriority(myList, item, newPriority):
    # our implementation makes use of a priority queue, this method allows us to update the priority of a given element
    # in the form (priority, station)
    for element in myList:
        if element[1] == item:
            new_element = (newPriority, element[1])  # create a new element with the new priority
            element = new_element  # replace our old element with the new element with the correct priority
            return


def haversine(fromS, toS):
    # this allows us to calculate the distance between two stations, taking into account the Earth's curvature.
    radius = 3958.8  # radius of earth in miles

    # access latitude and longitude of our stations and stores them in variables
    long1 = float(fromS.long)
    lat1 = float(fromS.lat)
    long2 = float(toS.long)
    lat2 = float(toS.lat)

    # map our latitudes and longitudes to radians
    long1, lat1, long2, lat2 = map(radians, [long1, lat1, long2, lat2])

    longDiff = long2 - long1
    latDiff = lat2 - lat1

    # haversine formula for the distance of two points given their latitudes and longitudes
    a = sin(latDiff / 2) ** 2 + cos(lat1) * cos(lat2) * sin(longDiff / 2) ** 2
    c = 2 * asin(sqrt(a))

    miles = radius * c
    return round(miles, 4)


Use the cell below to implement the requested API.

In [3]:
import csv

class LondonRailwayMapper(AbstractLondonRailwayMapper):
    
    def __init__(self):
        # our mapper will make use of a tubeMap (represents a graph)
        super().__init__()
        self.tubeMap = TubeMap()           
     
    
        
    def loadStationsAndLines(self):
        with open('londonstations.csv') as stations_file:  # open londonstations.csv file
            stations_reader = list(csv.reader(stations_file))
            count = 0
            # for each row in our file, creates a new station with a given  ID, name, latitude and longitude and
            # appends it to our list of stations. It also creates an adjacency list for each station.
            for row in stations_reader:
                if count == 0:
                    count += 1
                    continue
                station_id = count - 1
                station_name = row[0]
                station_lat = row[1]
                station_long = row[2]
                new_station = Station(station_id, station_name, station_lat, station_long)
                self.tubeMap.stations.append(new_station)
                self.tubeMap.adj.append([])
                count += 1

        with open('londonrailwaylines.csv') as lines_file:  # open londonrailwaylines.csv file
            lines_reader = csv.reader(lines_file)
            count = 0
            # for each row in the file, creates a new edge joining adjacent stations, the edge also stores the distance
            # between the stations
            for row in lines_reader:
                if count == 0:
                    count += 1
                    continue
                id1 = self.tubeMap.idFromName(row[1])
                id2 = self.tubeMap.idFromName(row[2])
                self.tubeMap.addEdge(id1, id2)  # adds edge to both stations' adjacency list
    
    

    def minStops(self, fromS, toS):
        numStops = -1

        # create a list with the distance from each station to our starting station.
        distToSource = [-1 for station in range(len(self.tubeMap.adj))]

        # get the id of our stations from their names
        id_fromS = self.tubeMap.idFromName(fromS)
        id_toS = self.tubeMap.idFromName(toS)

        q = deque()  # we will use a deque for cheap push and pop operations (constant time)
        q.append(id_fromS)
        distToSource[id_fromS] = 0

        # breadth-first search algorithm to calculate the minimum number of stops (unweighted graph)
        while len(q) != 0 and distToSource[id_toS] == -1:  # ensures we stop when we calculate the desired distance.
            v = q.popleft()  # pops the next station in the queue
            for edge in self.tubeMap.adj[v]:  # iterating through the edges in its adjacency list
                if distToSource[edge.idTo] == -1:  # only do something if the other station hasn't been visited yet.
                    q.append(edge.idTo)  # adds stations that haven't been visited yet
                    distToSource[edge.idTo] = distToSource[v] + 1  # calculates the distance to source of station

        numStops = distToSource[id_toS]

        return numStops   
    
    
    
    def minDistance(self, fromS, toS):
        minDistance = -1.0

        # starting distance from every station to the source is +infinity for the purpose of our algorithm
        distToSource = [inf for v in range(len(self.tubeMap.adj))]

        # gets IDs of both our source and destination stations
        fromID = self.tubeMap.idFromName(fromS)
        toID = self.tubeMap.idFromName(toS)

        # distance from the source to the source is logically 0
        distToSource[fromID] = 0

        # for this algorithm, we will be using a MINIMUM priority-queue, using a list and methods imported from heapq
        # elements in our priority queue will be couples, with the first element being the provisional distance to
        # the source, and the second element being the ID of our station.
        pq = []
        heapq.heappush(pq, (0, fromID))  # push our starting station with a distance 0

        # implementation of Dijkstra's algorithm to determine distance between two nodes in a weighted graph
        while len(pq) != 0:  # while our pq is not empty, keep running
            v = heapq.heappop(pq)[1]  # v will be the ID of the station that has a current lowest distance
            # for every edge in the adjacency list of v, update the distance to the source of the stations at the other
            # end of the edge if their current distance to the source is greater than the distance of v + the weight
            # of the edge
            for edge in self.tubeMap.adj[v]:
                w = edge.idTo
                if distToSource[w] > distToSource[v] + edge.weight:
                    distToSource[w] = distToSource[v] + edge.weight
                    if contains(pq, w):
                        updatePriority(pq, w, distToSource[w])  # updates the distance of a station
                    else:
                        heapq.heappush(pq, (distToSource[w], w))

        minDistance = distToSource[toID]  # we will return the distance to the source from our destination station
        return minDistance
    
    
    
    
    def newRailwayLine(self, inputList):
        inputList_noDuplicates = list(set(inputList))  # remove possible duplicates from the inputList
        outputList = []  # will contain the stations of our new line in order
        stationList = []  # stores the station object of each of the station names in the inputList
        maximum_latitude = -inf
        startingStation = None

        for stationName in inputList_noDuplicates:  # for each of the station names in our inputList
            station = self.tubeMap.stationFromName(stationName)  # get station object from the station name
            stationList.append(station)  # add station name to our list of stations

            # if the latitude of the station is greater than the maximum latitude, set the starting station
            # as the current station and update the maximum latitude
            if float(station.lat) > maximum_latitude:
                startingStation = station
                maximum_latitude = float(station.lat)

        # starting station in outputList will be the one with greatest latitude
        outputList.append(startingStation.name)
        lastStation = startingStation

        # nearest-neighbour approach to solve the given variation of TSP.
        # while the output list doesn't contain all the stations in the input list:
        while len(outputList) != len(inputList_noDuplicates):
            minimumDistance = inf
            for station in stationList:
                if station.name in outputList:
                    continue
                else:
                    # calculate distance between current station and all other stations in our list.
                    # the next station in our outputList will be the one with the least distance to our current station.
                    distance = haversine(lastStation, station)
                    if distance < minimumDistance:
                        minimumDistance = distance
                        nextStation = station
            outputList.append(nextStation.name)
            lastStation = nextStation

        return outputList

Use the cell below for all python code needed to test the `LondonRailwayMapper` class above.

In [4]:
import timeit
import random
import csv
from collections import deque
from math import radians, cos, sin, asin, sqrt, inf
import heapq

# ADD YOUR TEST CODE HERE

# testing time taken to load stations and creating graph
totalTime = 0
timesRun = 20
for i in range(timesRun):
    mapper = LondonRailwayMapper()
    starttime = timeit.default_timer()
    mapper.loadStationsAndLines()
    endtime = timeit.default_timer()
    totalTime = totalTime + (endtime - starttime)

print("\n (loadingStationsAndLines) Average runtime for " + str(timesRun) + " times ran = " + str(totalTime / timesRun)
      + " seconds")

# testing minStops method
totalTime = 0
timesRun = 20
for i in range(timesRun):
    mapper = LondonRailwayMapper()
    mapper.loadStationsAndLines()
    id1 = random.randint(0, len(mapper.tubeMap.stations) - 1)
    id2 = random.randint(0, len(mapper.tubeMap.stations) - 1)
    station1Name = mapper.tubeMap.stations[id1].name
    station2Name = mapper.tubeMap.stations[id2].name
    starttime = timeit.default_timer()
    mapper.minStops(station1Name, station2Name)
    endtime = timeit.default_timer()
    totalTime = totalTime + (endtime - starttime)

print("\n (minStops) Average runtime for " + str(timesRun) + " times ran = " + str(totalTime / timesRun)
      + " seconds")

# testing minDistance method
totalTime = 0
timesRun = 20
for i in range(timesRun):
    mapper = LondonRailwayMapper()
    mapper.loadStationsAndLines()
    id1 = random.randint(0, len(mapper.tubeMap.stations) - 1)
    id2 = random.randint(0, len(mapper.tubeMap.stations) - 1)
    station1Name = mapper.tubeMap.stations[id1].name
    station2Name = mapper.tubeMap.stations[id2].name
    starttime = timeit.default_timer()
    mapper.minDistance(station1Name, station2Name)
    endtime = timeit.default_timer()
    totalTime = totalTime + (endtime - starttime)

print("\n (minDistance) Average runtime for " + str(timesRun) + " times ran = " + str(totalTime / timesRun)
      + " seconds\n")

# testing newRailwayLine method

timesRun = 10
sampleSizes = [5, 10, 20, 50, 100, 200]

for sampleSize in sampleSizes:
    totalTime = 0
    for i in range(timesRun):
        mapper = LondonRailwayMapper()
        mapper.loadStationsAndLines()
        IDs = [random.randint(0, len(mapper.tubeMap.stations) - 1) for _ in range(sampleSize)]
        inputList = [mapper.tubeMap.stations[ID].name for ID in IDs]
        starttime = timeit.default_timer()
        mapper.newRailwayLine(inputList)
        endtime = timeit.default_timer()
        totalTime = totalTime + (endtime - starttime)
        # print("iteration number %i completed" % (i+1))

    print(" (newRailwayLine) Average runtime for " + str(timesRun) + " times ran for a sample size of " +
          str(sampleSize) + " = " + str(totalTime / timesRun) + " seconds\n")




 (loadingStationsAndLines) Average runtime for 20 times ran = 0.014718434999999997 seconds

 (minStops) Average runtime for 20 times ran = 0.00020970500000001558 seconds

 (minDistance) Average runtime for 20 times ran = 0.0020449900000000243 seconds

 (newRailwayLine) Average runtime for 10 times ran for a sample size of 5 = 0.00010487999999995168 seconds

 (newRailwayLine) Average runtime for 10 times ran for a sample size of 10 = 0.00023009999999996645 seconds

 (newRailwayLine) Average runtime for 10 times ran for a sample size of 20 = 0.0007972400000000768 seconds

 (newRailwayLine) Average runtime for 10 times ran for a sample size of 50 = 0.0050061300000000305 seconds

 (newRailwayLine) Average runtime for 10 times ran for a sample size of 100 = 0.01995151999999987 seconds

 (newRailwayLine) Average runtime for 10 times ran for a sample size of 200 = 0.08012188999999985 seconds



The cell below exemplifies the test code I will invoke on your submission. Do NOT modify it. 

In [5]:
# DO NOT MODIFY THIS CELL

import timeit

testMapper = LondonRailwayMapper()

#
# testing the loadStationsAndLines() API 
#
starttime = timeit.default_timer()
testMapper.loadStationsAndLines()
endtime = timeit.default_timer()
print("\nExecution time to load:", round(endtime-starttime,3))

#
# testing the minStops() and minStops() API on a sample of from/to station pairs  
#
fromList = ["Baker Street", "Epping", "Canonbury", "Vauxhall"]
toList = ["North Wembley", "Belsize Park", "Balham", "Leytonstone"]

for i in range(len(fromList)):
    starttime = timeit.default_timer()
    stops = testMapper.minStops(fromList[i], toList[i])
    endtime = timeit.default_timer()
    print("\nExecution time minStops:", round(endtime-starttime,3))

    starttime = timeit.default_timer()
    dist = testMapper.minStops(fromList[i], toList[i])
    endtime = timeit.default_timer()
    print("Execution time minDistance:", round(endtime-starttime,3))

    print("From", fromList[i], "to", toList[i], "in", stops, "stops and", dist, "miles")  
    
#
# testing the newRailwayLine() API on a small list of stations  
#
stationsList = ["Queens Park", "Chigwell", "Moorgate", "Swiss Cottage", "Liverpool Street", "Highgate"]

starttime = timeit.default_timer()
newLine = testMapper.newRailwayLine(stationsList)
endtime = timeit.default_timer()

print("\n\nStation list", stationsList)
print("New station line", newLine)
print("Total track length from", newLine[0], "to", newLine[len(newLine)-1], ":", testMapper.minDistance(newLine[0], newLine[len(newLine)-1]), "miles")
print("Execution time newLine:", round(endtime-starttime,3))

#
# testing the newRailwayLine() API on a big list of stations  
#
stationsList = ["Abbey Road", "Barbican", "Bethnal Green", "Cambridge Heath", "Covent Garden", "Dollis Hill", "East Finchley", "Finchley Road and Frognal", "Great Portland Street", "Hackney Wick", "Isleworth", "Kentish Town West", "Leyton", "Marble Arch", "North Wembley", "Old Street", "Pimlico", "Queens Park", "Richmond", "Shepherds Bush", "Tottenham Hale", "Uxbridge", "Vauxhall", "Wapping"]

starttime = timeit.default_timer()
newLine = testMapper.newRailwayLine(stationsList)
endtime = timeit.default_timer()

print("\n\nStation list", stationsList)
print("New station line", newLine)
print("Total track length from", newLine[0], "to", newLine[len(newLine)-1], ":", testMapper.minDistance(newLine[0], newLine[len(newLine)-1]), "miles")
print("Execution time newLine:", round(endtime-starttime,3))


Execution time to load: 0.018

Execution time minStops: 0.0
Execution time minDistance: 0.0
From Baker Street to North Wembley in 6 stops and 6 miles

Execution time minStops: 0.0
Execution time minDistance: 0.0
From Epping to Belsize Park in 17 stops and 17 miles

Execution time minStops: 0.0
Execution time minDistance: 0.0
From Canonbury to Balham in 10 stops and 10 miles

Execution time minStops: 0.0
Execution time minDistance: 0.0
From Vauxhall to Leytonstone in 6 stops and 6 miles


Station list ['Queens Park', 'Chigwell', 'Moorgate', 'Swiss Cottage', 'Liverpool Street', 'Highgate']
New station line ['Chigwell', 'Liverpool Street', 'Moorgate', 'Swiss Cottage', 'Queens Park', 'Highgate']
Total track length from Chigwell to Highgate : 18.590700000000005 miles
Execution time newLine: 0.0


Station list ['Abbey Road', 'Barbican', 'Bethnal Green', 'Cambridge Heath', 'Covent Garden', 'Dollis Hill', 'East Finchley', 'Finchley Road and Frognal', 'Great Portland Street', 'Hackney Wick', '